In [1]:
#Module we will use down the road
from __future__ import print_function
import numpy as np
import tensorflow as tf
import pickle

In [2]:
#Load the data
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [20]:
train_labels

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32)

In [3]:
#Reformat the data

image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
#Let's first run a multinomial logistic regressionwith a simple gradient descent
#subset the training data for faster trunaround
train_subset = 10000

graph = tf.Graph()
with graph.as_default():
    
    #remeber we are training a simple gradient descent -> no placeholder
    #the training, validation and test sets here are attached to the graph
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_validation_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #Variables: these are the parameters of the model;
    weights = tf.Variable(tf.truncated_normal([image_size*image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    
    #Training
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels = tf_train_labels, logits = logits))
    
    #Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    
    #Predictions
    train_predictions = tf.nn.softmax(logits)
    valid_predictions = tf.nn.softmax(
    tf.matmul(tf_validation_dataset, weights)+ biases)
    test_predictions = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [5]:
#Let's run this coputation and iterate
num_steps = 801
def accuracy(predictions, labels):
    return(100.0*np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))/ len(predictions))

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in np.arange(num_steps):
        _, l, predictions = session.run([optimizer, loss, train_predictions])
        if(step%100 == 0):
            print("Loss at step {0} : {1}".format(step, l))
            print("Training accuracy : %.2f%%"% accuracy(predictions, train_labels[:train_subset, :]))
            print("Validation accuracy : %.1f%%" % accuracy(valid_predictions.eval(), valid_labels))
    print("test accuracy : %.1f%%"%accuracy(test_predictions.eval() , test_labels))

Initialized
Loss at step 0 : 16.361934661865234
Training accuracy : 6.99%
Validation accuracy : 11.1%
Loss at step 100 : 2.3478381633758545
Training accuracy : 72.13%
Validation accuracy : 70.3%
Loss at step 200 : 1.9032552242279053
Training accuracy : 74.90%
Validation accuracy : 72.3%
Loss at step 300 : 1.650117039680481
Training accuracy : 76.46%
Validation accuracy : 73.2%
Loss at step 400 : 1.475570559501648
Training accuracy : 77.29%
Validation accuracy : 73.8%
Loss at step 500 : 1.3465306758880615
Training accuracy : 77.88%
Validation accuracy : 74.2%
Loss at step 600 : 1.2461357116699219
Training accuracy : 78.58%
Validation accuracy : 74.5%
Loss at step 700 : 1.1651231050491333
Training accuracy : 79.11%
Validation accuracy : 74.7%
Loss at step 800 : 1.098054051399231
Training accuracy : 79.66%
Validation accuracy : 74.9%
test accuracy : 82.3%


In [6]:
#Stochastic Gradient Descent
batch_size = 128
graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape= (batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape = (batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #variables
    weights = tf.Variable(tf.truncated_normal([image_size*image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    #Training
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels = tf_train_labels, logits = logits))
    
    #Optimization
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    #predictions
    train_predictions = tf.nn.softmax(logits)
    valid_predictions = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_predictions = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [7]:
#train
num_steps = 3001
with tf.Session(graph = graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in np.arange(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0]- batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset:batch_data, tf_train_labels:batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_predictions], feed_dict = feed_dict)
        if step%500 == 0:
            print("Minibatch loss at {0}: {1}".format(step, l))
            print("Minibatch accuray : %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                        valid_predictions.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_predictions.eval(), test_labels))

Initialized
Minibatch loss at 0: 18.662269592285156
Minibatch accuray : 7.8%
Validation accuracy: 9.8%
Minibatch loss at 500: 1.1725246906280518
Minibatch accuray : 78.1%
Validation accuracy: 75.9%
Minibatch loss at 1000: 1.3263218402862549
Minibatch accuray : 74.2%
Validation accuracy: 77.0%
Minibatch loss at 1500: 0.7739146947860718
Minibatch accuray : 83.6%
Validation accuracy: 77.5%
Minibatch loss at 2000: 0.8434993028640747
Minibatch accuray : 78.9%
Validation accuracy: 77.5%
Minibatch loss at 2500: 1.032373309135437
Minibatch accuray : 79.7%
Validation accuracy: 78.5%
Minibatch loss at 3000: 1.106473684310913
Minibatch accuray : 78.9%
Validation accuracy: 79.0%
Test accuracy: 86.3%


In [13]:
#1 layer Neural Network using rectified linear units and 1024 hidden nodes
batch_size = 128
num_hidden_units = 1024
graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape = (batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape = (batch_size, num_labels))
    tf_validation_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #variables
    weights1 = tf.Variable(tf.truncated_normal([image_size*image_size, num_hidden_units]))
    biases1 = tf.Variable(tf.zeros([num_hidden_units]))
    weights2 = tf.Variable(tf.truncated_normal([num_hidden_units, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))
    
    #Training
    hidden1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
    logits = tf.matmul(hidden1, weights2) + biases2
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = tf_train_labels, logits = logits))
    
    #Optimization
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    #Predictions
    train_predictions = tf.nn.softmax(logits)
    valid_hidden1 = tf.nn.relu(tf.matmul(tf_validation_dataset, weights1) + biases1)
    valid_predictions = tf.nn.softmax(tf.matmul(valid_hidden1, weights2) + biases2)
    test_hidden1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1 )
    test_predictions = tf.nn.softmax(tf.matmul(test_hidden1, weights2) + biases2)

In [14]:
#train
num_steps = 3001
with tf.Session(graph = graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in np.arange(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0]- batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset:batch_data, tf_train_labels:batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_predictions], feed_dict = feed_dict)
        if step%500 == 0:
            print("Minibatch loss at {0}: {1}".format(step, l))
            print("Minibatch accuray : %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                        valid_predictions.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_predictions.eval(), test_labels))

Initialized
Minibatch loss at 0: 332.98333740234375
Minibatch accuray : 10.9%
Validation accuracy: 24.2%
Minibatch loss at 500: 25.510574340820312
Minibatch accuray : 77.3%
Validation accuracy: 80.2%
Minibatch loss at 1000: 12.710750579833984
Minibatch accuray : 78.9%
Validation accuracy: 81.2%
Minibatch loss at 1500: 7.755655288696289
Minibatch accuray : 87.5%
Validation accuracy: 81.0%
Minibatch loss at 2000: 1.66506028175354
Minibatch accuray : 89.1%
Validation accuracy: 82.1%
Minibatch loss at 2500: 4.591264724731445
Minibatch accuray : 79.7%
Validation accuracy: 81.6%
Minibatch loss at 3000: 1.7095650434494019
Minibatch accuray : 85.2%
Validation accuracy: 82.3%
Test accuracy: 89.8%
